# Test 2


### Рубенчик Марк Ильич, 631 гр.

We are going to work with the following dataset: fluid current in a tube.
Some statistics are collected for dataset, incl. mean, skewness, kurtosis, etc. We are predicting flow rate ('tohn/hour'). We need to build confidence and predictive intervals.

In [1]:
%matplotlib inline

import numpy as np
from sklearn import datasets, linear_model, preprocessing, model_selection
from sklearn.utils import shuffle
import matplotlib.pyplot as plt
import pandas as pd

import statsmodels.stats.api as sms

In [2]:
data = pd.read_csv('exxsol_data.csv', sep=';', header=(0))

print(data.columns.values)

data.head()

['mean' 'std' 'skew' 'kurt' 'RMS' 'crest' 'freq_peak' 'shan' 'perm' 'temp'
 'tohn/hour']


,mean,std,skew,kurt,RMS,crest,freq_peak,shan,perm,temp,tohn/hour
0,0.025639,0.946336,-0.034895,-1.491265,0.946674,0.888424,526.01052,9.584955,1.000464,21.14,3.49
1,0.025639,0.946336,-0.034895,-1.491265,0.946674,0.888424,526.01052,9.584955,1.000464,21.14,3.49
2,0.016856,0.952439,-0.037338,-1.504434,0.952579,0.943467,530.01060,9.402718,0.991088,21.14,3.49
3,0.016856,0.952439,-0.037338,-1.504434,0.952579,0.943467,530.01060,9.402718,0.991088,21.14,3.49
4,0.031998,0.972590,-0.056116,-1.546518,0.973106,0.921600,256.00512,9.020444,0.973701,21.14,3.56


In [3]:
y = data['tohn/hour']
freq_temp = data[['freq_peak','temp']]

Physics tells us that flow rate is a function of a frequency peak and temperature.

In [4]:
freq_temp, y = shuffle(freq_temp, y)

In [5]:
# split data into training and testing sets
#from sklearn.model_selection import train_test_split
#train_freq, test_freq, train_y, test_y = train_test_split(freq, y, train_size=0.7, random_state=2)

model_lr = linear_model.LinearRegression()
pred = model_selection.cross_val_predict(model_lr,freq_temp, y.ravel(), cv=20)
scores_r2 = model_selection.cross_val_score(model_lr, freq_temp, y, scoring='r2',cv=20)

In [6]:
model_elastic = linear_model.ElasticNetCV(cv=10, max_iter=3000, n_alphas=10)
pred_elastic = model_selection.cross_val_predict(model_elastic, freq_temp, y.ravel(), cv=20)
scores_r2_elastic = model_selection.cross_val_score(model_elastic, freq_temp, y.ravel(), scoring='r2',cv=20)

## Q0: Evaluation based on predictions. Model selection based on $R^2$ metric. Can we choose any model based on 95% confidence interval?

Чтобы сравнить $R^2$-оценки, мы будем использовать t-тест для разности средних. Допустим, что у нас есть две модели с истинными значениями $R^2_1$ и $R^2_2$ соответственно. Мы предполагаем, что вторая модель с регуляризацией лучше, чем первая модель, потому что она может предотвратить переобучение. 

Сформулируем гипотезы:

$H_0: R^2_1 = R^2_2$,

$H_1: R^2_1 < R^2_2$.

In [7]:
alpha = 0.05

In [8]:
Comp_Means = sms.CompareMeans(sms.DescrStatsW(scores_r2), sms.DescrStatsW(scores_r2_elastic))
Comp_Means.tconfint_diff(alternative = 'larger', usevar='pooled', alpha = alpha)

(-0.012150965640723425, inf)

Как мы видим, $0$ попадает в этот доверительный интервал. Поэтому мы не можем утверждать, что значение $R^2$ у этих моделей отличаются друг от друга на уровне значимости $\alpha = 0.05$.


## Q1:  Goodness of fit. Implement AIC, AICc and compare models using these criteria

Обе модели - регрессии, построенные на $2$ факторах, поэтому в моделях будет $k = 3$ параметра:

In [9]:
def AIC_crit(y, y_pred, n_params):
    n = y_pred.shape[0]
    rss = np.sum((y_pred - y)**2)
    return float(2*n_params + n * np.log(rss))

In [12]:
AIC = AIC_crit(y, pred, 3)
print("AIC for linear model: ",AIC)

AIC for linear model:  3322.4686695808296


In [14]:
AIC_elastic = AIC_crit(y, pred_elastic, 3)
print("AIC for elastic linear model: ",AIC_elastic)

AIC for elastic linear model:  3497.4976003328957


Как видно, первая модель более предпочтительна, чем вторая.

In [15]:
def AIC_crit_c(y, y_pred, n_params):
    n = y_pred.shape[0]
    rss = np.sum((y_pred - y)**2)
    return float((n + n_params)/(n - n_params - 2) +  np.log(rss/n))

In [17]:
AICc = AIC_crit_c(y, pred, 3)
print("AICc for linear model: ",AICc)

AICc for linear model:  -0.8017571585610268


In [19]:
AICc_elastic = AIC_crit_c(y, pred_elastic, 3)
print("AICc for elastic linear model: ",AICc_elastic)

AICc for elastic linear model:  -0.5517158289152184


Как видно, первая модель регрессии более предпочтительна, чем вторая, но не сильно. Значения показателей отличаются, но не сильно в порядке 9 процентов для AIC и 30 процентов для AICc

## Q2:  More advanced. Implement Mallow’s Cₚ criterion:

Wiki defenition of Mallows $C_p$:

$C_{p}={\frac {1}{n}}(\operatorname{RSS} +2p{\hat\sigma}^{2})$

In [20]:
def Mallow_c(y, y_pred, n_params):
    n = y_pred.shape[0]
    rss = np.sum((y_pred - y)**2)
    sigma = y_pred.std()
    return float((1/n)*(rss + 2*(n_params - 1)*sigma**2))

In [22]:
mallow = Mallow_c(y, pred, 3)
print("Mallow for linear model: ",mallow)

Mallow for linear model:  0.16786678290258522


In [23]:
mallow_elastic = Mallow_c(y, pred_elastic, 3)
print("Mallow for elastic linear model: ",mallow_elastic)

Mallow for elastic linear model:  0.213687121709552


## Q3: There is also BIC criteria. Can you recommend when to use AIC vs BIC and vice versa?

AIC и BIC предназначены для разных задач и случаев.

AIC используется в ситуации, когда не обязательно предполагается существование какой-либо модели. Вместо этого есть набор эффектов и мы можем получить значения ошибок прогноза. По мере увеличения размера выборки применимость AIC также увеличивается, так как увеличивается влияние меньших эффектов.


С другой стороны, BIC предполагает, что модель существует и должна быть выбрана. В БИК есть функция выделения одной модели по мере роста числа дел, в АИК нет. По мере увеличения размера выборки AIC выбирает параметрически большие модели, чем BIC.